# Payment 관련 feature engineering

In [2]:
# train data
import pandas as pd

In [6]:
payment_df = pd.read_csv('C:/Users/CPB06GameN/Desktop/빅콘/2019_빅콘테스트_챔피언스리그_데이터/train_payment.csv')
payment_df.shape
#(66432, 3)
payment_df.tail()

,day,acc_id,amount_spent
66427,1,13809,1.056123
66428,2,13809,0.697041
66429,17,13809,0.117347
66430,15,17325,0.011735
66431,5,27258,0.035204


### 1. 이해는 잘 안되었지만 날짜별로 지불한 금액을 파악한다. (평균으로 한다는 것인지가 헷갈림)_뭐를 한 것일까...

In [28]:
payment_amount = payment_df.pivot("acc_id", "day", "amount_spent")
payment_amount.rename(columns={1:"pay_amount_w1", 2:"pay_amount_w2", 3:"pay_amount_w3", 
                               4:"pay_amount_w4", 5:"pay_amount_w5", 6:"pay_amount_w6",
                               7:"pay_amount_w7", 8:"pay_amount_w8", 9:"pay_amount_w9", 
                               10:"pay_amount_w10", 11:"pay_amount_w11", 12:"pay_amount_w12", 
                               13:"pay_amount_w13", 14:"pay_amount_w14", 15:"pay_amount_w15",
                               16:"pay_amount_w16", 17:"pay_amount_w17", 18:"pay_amount_w18",
                               19:"pay_amount_w19",  20:"pay_amount_w20", 21:"pay_amount_w21", 
                               22:"pay_amount_w22", 23:"pay_amount_w23", 24:"pay_amount_w24",
                               25:"pay_amount_w25", 26:"pay_amount_w26", 27:"pay_amount_w27", 
                               28:"pay_amount_w28" }, inplace=True)
payment_amount.reset_index(inplace=True)
payment_amount = payment_amount.set_index('acc_id')
payment_amount.tail()

day,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_w9,pay_amount_w10,...,pay_amount_w19,pay_amount_w20,pay_amount_w21,pay_amount_w22,pay_amount_w23,pay_amount_w24,pay_amount_w25,pay_amount_w26,pay_amount_w27,pay_amount_w28
acc_id,,,,,,,,,,,,,,,,,,,,,
130447,1.056123,NaN,NaN,NaN,NaN,NaN,NaN,1.056123,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011735,NaN
130449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.011735,0.348521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130470,NaN,NaN,0.035204,NaN,NaN,NaN,NaN,NaN,NaN,1.228623,...,NaN,NaN,0.129082,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130473,0.127908,NaN,NaN,0.035204,0.105612,0.070408,0.880103,NaN,NaN,0.117347,...,0.281633,0.211225,0.880103,NaN,0.070408,NaN,NaN,NaN,NaN,NaN


### 2. 금액을 지불한 적이 있었던 날짜 카운트 변수 생성(몇번 질렀는지)

In [34]:
payment_day = pd.read_csv('C:/Users/CPB06GameN/Desktop/빅콘/2019_빅콘테스트_챔피언스리그_데이터/train_payment.csv')
cnt_payment_day = payment_day.loc[:,['acc_id','amount_spent']].groupby('acc_id').size().reset_index(name="cnt_payment_day")
cnt_payment_day.tail()

,acc_id,cnt_payment_day
23721,130447,5
23722,130449,1
23723,130469,2
23724,130470,3
23725,130473,12


### 3. 그렇다면 마지막으로 지른 것은 언제인지

유저가 마지막으로 payment를 했던 주가 1-8주 중 몇 번째 주인지 나타내는 변수 생성
(라고 설명하는데 이해 노노)

In [36]:
result = payment_day.loc[:,['acc_id','amount_spent']].groupby('acc_id').agg(['max']).reset_index()
last_payment_day = pd.DataFrame(columns=["acc_id", "last_payment_day"])
last_payment_day["acc_id"] = result["acc_id"]
last_payment_day["last_payment_day"] = result["amount_spent"]["max"]
last_payment_day.tail()

,acc_id,last_payment_day
23721,130447,1.056123
23722,130449,0.348521
23723,130469,0.348521
23724,130470,1.228623
23725,130473,0.880103


### 4. 일별 금액의 최고값과 전체 금액 변수 생성

유저 최고 금액과 생존 기간 중 금액의 합

In [38]:
result = payment_df.loc[:,["acc_id","amount_spent"]].groupby("acc_id").agg(["max","sum"]).reset_index()
payment = pd.DataFrame(columns=["acc_id", "max_payment_amount", "total_payment_amount"])
payment["acc_id"] = result["acc_id"]
payment["max_payment_amount"] = result["amount_spent"]["max"]
payment["total_payment_amount"] = result["amount_spent"]["sum"]
payment.tail()

,acc_id,max_payment_amount,total_payment_amount
23721,130447,1.056123,3.215308
23722,130449,0.348521,0.348521
23723,130469,0.348521,0.360255
23724,130470,1.228623,1.392909
23725,130473,0.880103,2.955971


### 5. day payment의 표준 편차 변수를 생성

유저의 주별 payment amount의 표준편차를 구해, 유저가 주별로 고르게 결제했는지 몰아서 결제했는지를 나타내는 변수 생성

In [39]:
payment_amount['pay_amount_std']=payment_amount.loc[:,"pay_amount_w1":"pay_amount_w28"].std(axis =1 )
payment_amount.fillna(0, inplace=True)
payment_amount.tail()

day,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_w9,pay_amount_w10,...,pay_amount_w20,pay_amount_w21,pay_amount_w22,pay_amount_w23,pay_amount_w24,pay_amount_w25,pay_amount_w26,pay_amount_w27,pay_amount_w28,pay_amount_std
acc_id,,,,,,,,,,,,,,,,,,,,,
130447,1.056123,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.056123,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.011735,0.0,0.565669
130449,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
130469,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.011735,0.348521,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.238144
130470,0.000000,0.0,0.035204,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.228623,...,0.000000,0.129082,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.663583
130473,0.127908,0.0,0.000000,0.035204,0.105612,0.070408,0.880103,0.000000,0.0,0.117347,...,0.211225,0.880103,0.0,0.070408,0.0,0.0,0.0,0.000000,0.0,0.303539


### 6. 하나로 합쳐보자!!

In [42]:
label = pd.read_csv('C:/Users/CPB06GameN/Desktop/빅콘/2019_빅콘테스트_챔피언스리그_데이터/train_label.csv')
label.tail()

,acc_id,survival_time,amount_spent
39995,13809,64,0.302485
39996,83195,64,0.005119
39997,17325,64,0.047182
39998,99716,39,0.000000
39999,27258,64,0.006661


In [45]:
result = pd.merge(label, payment_amount, how='inner',on='acc_id').fillna(value=0)
result = pd.merge(result, cnt_payment_day, how='inner',on='acc_id').fillna(value=0)
result = pd.merge(result, last_payment_day, how='inner',on='acc_id')
result = pd.merge(result, payment, how='inner',on='acc_id')
result = pd.merge(result, payment_amount, how='inner',on='acc_id')
result.tail()

,acc_id,survival_time,amount_spent_x,pay_amount_w1_x,pay_amount_w2_x,pay_amount_w3_x,pay_amount_w4_x,pay_amount_w5_x,pay_amount_w6_x,pay_amount_w7_x,...,pay_amount_w20_y,pay_amount_w21_y,pay_amount_w22_y,pay_amount_w23_y,pay_amount_w24_y,pay_amount_w25_y,pay_amount_w26_y,pay_amount_w27_y,pay_amount_w28_y,pay_amount_std_y
66427,13809,64,0.302485,1.056123,0.697041,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473689
66428,13809,64,0.302485,1.056123,0.697041,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473689
66429,13809,64,0.302485,1.056123,0.697041,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473689
66430,17325,64,0.047182,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
66431,27258,64,0.006661,0.000000,0.000000,0.0,0.0,0.035204,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [46]:
result.describe(include="all")

,acc_id,survival_time,amount_spent_x,pay_amount_w1_x,pay_amount_w2_x,pay_amount_w3_x,pay_amount_w4_x,pay_amount_w5_x,pay_amount_w6_x,pay_amount_w7_x,...,pay_amount_w20_y,pay_amount_w21_y,pay_amount_w22_y,pay_amount_w23_y,pay_amount_w24_y,pay_amount_w25_y,pay_amount_w26_y,pay_amount_w27_y,pay_amount_w28_y,pay_amount_std_y
count,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,...,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000,66432.000000
mean,65352.577387,49.136034,0.212419,0.311661,0.126571,0.088678,0.074982,0.069603,0.069665,0.065333,...,0.106387,0.088331,0.096939,0.103409,0.093918,0.098471,0.093155,0.086905,0.097536,0.556866
std,37413.396510,21.333901,0.748565,0.809816,0.444176,0.349647,0.320346,0.293345,0.304483,0.287294,...,0.412221,0.375199,0.377795,0.406533,0.359813,0.386791,0.373966,0.347893,0.384021,0.595697
min,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32646.000000,30.000000,0.035797,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154828
50%,65709.000000,64.000000,0.113461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.409608
75%,97500.250000,64.000000,0.241087,0.035204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.743269
max,130473.000000,64.000000,39.412632,10.739599,6.106739,6.160718,7.435107,5.782861,5.805157,6.160718,...,5.922504,9.087353,8.096944,7.510209,8.683679,11.733528,10.443884,5.867351,6.864800,7.302794
